In [ ]:
#所需的库
import os
import re
import json,requests
import geopandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import datetime
import js2py

In [ ]:
#以时间为x轴，确诊人数为y轴来绘制折线图
alt.Chart(data).mark_line().encode(x='时间',y='确诊人数')

#读取本地csv数据
file_name = 'D:\123\Desktop\COVID-19-DataSet\全球国家每日历史数据（截止至2020-05-31）.csv'
chinaDayHisDf = pd.read_csv(file_name)

In [ ]:
#绘制以日期为x轴，以累计确诊人数为y轴的折线图
chart = alt.Chart(chinaDayHisDF).mark_line().encode(
    x='日期',
    y='累计确诊'
)

In [ ]:
#及时地迭代日期
def changeDate(dateStr):
    # 定义匹配规则，将'月'前面的数字提取并划分到group[0]
    # 将'日'前面的数字提取并划分到group[1]
    rex = r'([\d]+)月([\d]+)日'
    
    # 对dateStr进行匹配，并获取对应的月 日
    m = re.match(rex,dateStr)
    mon = m.groups()[0]
    day = m.groups()[1]
    
    # 构建初始化TimrStamp所需要的字符串
    str = '2020-%s-%s' % (mon,day)
    date = pd.to_datetime(str)
    return date

In [ ]:
#再一次绘制图表
chinaDayHisDF['日期'] = chinaDayHisDF['日期'].apply(changeDate)

In [ ]:
# 使用axis参数对轴进行定义，此处传进一个自定义的alt.Axis对象
alt.Chart(chinaDayHisDF).mark_line().encode(
    x=alt.X('日期'),y=alt.X('累计确诊',axis=alt.Axis(title='累计确诊人数',titleFontSize=15,titleColor='red'))
)

In [ ]:
# 创建可视化确诊和疑似确诊 Altair库需要的数据格式
data = {
    'date':chinaDayHisDF['日期'],
    'symbol':'累计确诊',
    'count':chinaDayHisDF['累计确诊']

} 
chinaConfAndSusDf = pd.DataFrame(data)

data = {
    'date':chinaDayHisDF['日期'],
    'symbol':'累计疑似',
    'count':chinaDayHisDF['累计疑似']
}
chinaConfAndSusDf = chinaConfAndSusDf.append(pd.DataFrame(data))

data = {
    'date':chinaDayHisDF['日期'],
    'symbol':'累计死亡',
    'count':chinaDayHisDF['累计死亡']
}
chinaConfAndSusDf = chinaConfAndSusDf.append(pd.DataFrame(data))

In [ ]:
#给不同曲线绘制出不同的颜色，并设置图表的长宽
color = alt.Color('symbol:N',legend=alt.Legend(title='图例'),
                  scale=alt.Scale(domain=['累计确诊','累计疑似','累计死亡'], range=['#FF4433','#FF9933','#112233']))

alt.Chart(chinaConfAndSusDf).mark_line().encode(
    x=alt.X('date',axis=alt.Axis(title='日期')),
    y=alt.Y('count:Q',axis=alt.Axis(title='累计确诊人数',titleFontSize=15,titleColor='red')),
    color=color
).properties(width=1200, height=350)

In [ ]:
nearest = alt.selection_multi(name='nearest', nearest=True, on='mouseover',fields=['date'],empty='none')
line = alt.Chart(chinaConfAndSusDf,width=800,height=300).mark_line().encode(
    x=alt.X('date', axis=alt.Axis(title='日期',titleFontSize=16,titlePadding=20,grid=True,labelFontSize=14)),
    y=alt.Y('count', axis=alt.Axis( title='人数',titleFontSize=16,titleColor='#FF9988',titlePadding=20)),
    color=alt.Color('symbol',legend=alt.Legend(title=None,titleFontSize=16,titleOpacity=0.5,titlePadding=20,labelFontSize=16,rowPadding=10,labelOpacity=0.5,offset=60),
                   scale=alt.Scale(domain=['累计确诊','累计疑似'], range=['#FF4433','#FF9933']))
)

points = line.mark_point(strokeWidth=5).encode(
    x=alt.X('date', axis=alt.Axis(title='日期',titleFontSize=16,titlePadding=20,grid=True,labelFontSize=14)),
    y=alt.Y('count', axis=alt.Axis( title='人数',titleFontSize=16,titleColor='#FF9988',titlePadding=20)),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
).add_selection(nearest) 

text = line.mark_text(align='left', dx=5, dy=-15,fontSize=15, fontWeight='bold').encode(text=alt.condition(nearest, 'count', alt.value(' ')),)
rules = alt.Chart(chinaConfAndSusDf).mark_rule(strokeWidth=2,color='gray',opacity=0.2).encode(x=alt.X('date')
).transform_filter(nearest)
alt.layer(line,rules,points,text)
nearest = alt.selection(name='nearest',type='single', nearest=True, on='mouseover',fields=['date'],empty='none')
selCur = alt.selection_multi(name='selCur',fields=['symbol'],bind='legend')

# color类可以为我们的曲线，图例设置不同的颜色，区分曲线，
color = alt.Color('symbol:N',legend=alt.Legend(title=None,titleFontSize=16,titleOpacity=0.5,titlePadding=20,labelFontSize=16,rowPadding=10,labelOpacity=0.5,offset=60),
                    scale=alt.Scale(domain=['累计确诊','累计疑似','累计死亡'], range=['#FF4433','#FF9933','#112233']))

# alt.Chart(data)画出图，这里data是我们需要绘制的数据，mark_line方法表明这是一个线图
line = alt.Chart(chinaConfAndSusDf).mark_line().encode(
    x=alt.X('monthdate(date):T', axis=alt.Axis(title='日期',titleFontSize=16,titlePadding=20,grid=True,labelFontSize=14)),
    y=alt.Y('count', axis=alt.Axis( title='人数',titleFontSize=16,titleColor='#FF9988',titlePadding=20)),
    # condition 跟selection配合使用，第一个值为selection，第二个值是True时的值，即被选中时的值，第三个Fasle值
    color=alt.condition(selCur,color,alt.value('lightgray')),
    strokeWidth=alt.condition(selCur,alt.value(5),alt.value(3))
)

# 将选择器挂载到我们的图上
selectors = alt.Chart(chinaConfAndSusDf).mark_point(strokeWidth=3).encode(
    x=alt.X('monthdate(date):T'),
    opacity=alt.value(0),
).add_selection(nearest)


points = line.mark_point(strokeWidth=5).encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
    size=alt.condition(selCur,alt.value(3),alt.value(0)),
).add_selection(selCur)

text = line.mark_text(align='left', dx=5, dy=-15,fontSize=15, fontWeight='bold').encode(
    text=alt.condition(nearest, 'count', alt.value(' ')),
    opacity=alt.condition(selCur, alt.value(1), alt.value(0)),
)

rules = alt.Chart(chinaConfAndSusDf).mark_rule(strokeWidth=2,color='gray',opacity=0.2).encode(
    x=alt.X('monthdate(date):T')).transform_filter(nearest)

alt.layer(line, selectors, points, rules, text).properties(
    width=1200, height=350,
    title= {
    "text": "COVID-19 中国 累计确诊人数和累计疑似人数折线图",
    "fontSize": 20,
    'offset':20
    }
).configure_view(stroke=None)

In [ ]:
file_name = 'D:\123\Desktop\COVID-19-DataSet\全球国家每日历史数据（截止至2020-05-31）.json'
with open('%s' % file_name,'r') as f:
    globalDict = json.loads(f.read())

globalCountryDf = pd.DataFrame()

for name in globalDict.keys():
    tempDf = pd.DataFrame(globalDict[name])
    tempDf['日期'] = tempDf['日期'].apply(changeDate)
    df = tempDf[tempDf['日期'] == np.max(tempDf['日期'])]
    d = {
        'date' : df['日期'],
        'name': name,
        'confirm': df['累计确诊']
    }
    globalCountryDf = globalCountryDf.append(pd.DataFrame(d))

globalPCountryDf = globalCountryDf.sort_values('confirm',ascending=False).head(20)
selBarCur = alt.selection_multi(name='selBarLegend',fields=['name'],bind='legend')
hoverSel = alt.selection(name='hoverSel',type='single', nearest=True, on='mouseover',
                        fields=['name'],empty='none')
color = alt.Color('name:N',scale=alt.Scale(
    domain=list(globalPCountryDf['name']),
    range=['#d62728','#9467bd','#e377c2','#ff7f0e','#1f77b4',
           '#8c564b','#17becf','#FF6666','#c49c94','#9edae5',
           '#c5b0d5','#dbbb88','#ff9896','#7f7f7f','#98df8a',
           '#2ca02c','#ffbb78','#bcbd22','#aec7e8','#f7b6d2']),
                  legend=alt.Legend(title='国家', titleFontSize=16,titleOpacity=0.5,titlePadding=20,labelFontSize=15,rowPadding=5,labelOpacity=0.8,offset=60,orient='left'))

bar = alt.Chart(globalPCountryDf).mark_bar(size=23).encode(
    x=alt.X('confirm',axis=alt.Axis(title='人数',titleFontSize=16,titlePadding=20,grid=True,labelFontSize=15)), 
    y=alt.Y('name:N', sort=list(globalPCountryDf),axis=alt.Axis(title='国家',titleFontSize=18,titlePadding=20,grid=True,tickSize=15,labelFontSize=15,labelPadding=2,labelAngle=-25)),
    color=alt.condition(selBarCur,color,alt.value('lightgray')),
    size=alt.condition(hoverSel,alt.value(28),alt.value(20)),
)

point = alt.Chart(globalPCountryDf).mark_point().encode(
    y=alt.Y('name',sort=list(globalPCountryDf)),
    opacity=alt.value(0)
).add_selection(hoverSel)

text = bar.mark_text(align='left', dx=10, dy=0,fontSize=15, fontWeight='bold').encode(
    text='confirm',
    opacity=alt.condition(selBarCur,alt.value(1),alt.value(0))
).add_selection (selBarCur)

alt.layer(bar,text,point).properties(
    width=1200,height=600,
    title= {
    "text": "COVID-19 海外各国（地区） 确诊人数柱状图",
    "fontSize": 20,
    'offset':20
  }
)

In [ ]:
color = alt.Color('confirm:N',legend=alt.Legend(title=None,titlePadding=20,rowPadding=-1,symbolStrokeWidth=0,symbolType='square',symbolSize=300,
                                                labelPadding=20,labelOpacity=0,clipHeight=12,
                                                titleFontSize=14,titleOpacity=0.5),scale=alt.Scale(scheme='yelloworangebrown'))

shape = alt.Chart().mark_geoshape(fill='lightblue')
geo = alt.Chart(world_gdf).mark_geoshape().encode(
    color = color
).project('naturalEarth1').properties(
    width=1200,height=600,
    title= {
    "text": "COVID-19 世界各国（地区） 确诊人数热度图",
    "fontSize": 20,
    'offset':20
    }
)

hover = alt.selection(type='single', on='mouseover', nearest=True,fields=['lat','lon'])
base = alt.Chart(world_gdf).encode(
    longitude='lon:Q',
    latitude='lat:Q',
)
point = base.mark_point().encode(
    color=alt.value('lightblue'),
    size=alt.condition(~hover, alt.value(30), alt.value(100)),
    tooltip=['cname', 'confirm'],
    opacity=alt.value(1)
).add_selection(hover)

layer = alt.layer(shape,geo,point)
layer.configure_view(stroke=None)